<a href="https://colab.research.google.com/github/shubham-kashyapi/CS6910-Deep-Learning-Assignment-4/blob/main/A4_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload() # Upload the files RBM_model.py and logistic_classifier.py

Saving logistic_classifier.py to logistic_classifier.py
Saving RBM_model.py to RBM_model.py


{'RBM_model.py': b'import numpy as np\nfrom tqdm import tqdm\n\nclass RBM:\n    def __init__(self, num_visible, num_hidden):\n        \'\'\'\n        Initializes the parameters of a RBM\n        Parameters:\n        num_visible - int, num_hidden - int\n        Returns:\n        None\n        \'\'\'\n        self.num_visible = num_visible\n        self.num_hidden = num_hidden\n        #######################################\n        # Random initialization of parameters\n        #######################################\n        self.W = np.random.normal(size = (self.num_hidden, self.num_visible))\n        self.b = np.random.normal(size = (1, self.num_visible))\n        self.c = np.random.normal(size = (self.num_hidden, 1))\n        return\n    \n    def check_data_format(self, input_data):\n        \'\'\'\n        Checks the format of visible variables\n        \n        Parameters:\n        input_data - 2d numpy array (dtype=float, size = (num_samples, num_visible))\n        Should cont

In [2]:
!pip install wandb
import numpy as np
from keras.datasets import fashion_mnist # Used only for loading the data
from RBM_model import RBM
from logistic_classifier import LogisticClassifier
import wandb
from tqdm.notebook import tqdm

     |████████████████████████████████| 1.8MB 5.0MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 174kB 36.1MB/s 
     |████████████████████████████████| 133kB 36.2MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=ea20c7b47f133f5cece3045be2f4ab09762a1f1e7f534bbcbc34f2fc61e970e0
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=0551768128f8352ed0e352eafd77f95d99d0fbc73e62130651d44c6b5e7db640
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [3]:
###############################
# Preparing the data
###############################
# Loading the pre-shuffled fashion mnist dataset
(X_fashion_train, y_fashion_train), (X_fashion_test, y_fashion_test) = fashion_mnist.load_data()
# Using only a part of the training data and splitting it into training and validation sets
X_train = (X_fashion_train.reshape(60000, 784)[:10000, :] > 127).astype(float)
y_train = y_fashion_train[:10000]
X_val = (X_fashion_train.reshape(60000, 784)[10000 : 12500, :] > 127).astype(float)
y_val = y_fashion_train[10000 : 12500]
# Using only a part of the test data
X_test = (X_fashion_test.reshape(10000, 784)[:2500, :] > 127).astype(float)
y_test = y_fashion_test[:2500]
# Checking the shapes
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

4423680/4422102 [==============================] - 0s 0us/step
(10000, 784) (10000,) (2500, 784) (2500,) (2500, 784) (2500,)


In [4]:
###################################################
# Listing the hyperparameters in a wandb config 
###################################################
sweep_config = {'name': 'random-test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'test_acc', 'goal': 'maximize'}
parameters_dict = {
                   'num_hidden_vars': {'values': [64, 128, 256]},     # n - mentioned in question 2
                   'k': {'values': [100, 200, 300]},                  # k - mentioned in question 2
                   'r': {'values': [10, 20, 30]},                     # r - mentioned in question 2
                   'num_visible_vars': {'values': [784]},
                   'num_classes': {'values': [10]},
                   'rbm_train_type': {'values': ['BGS']},
                   'num_epochs_rbm': {'values': [10]}, # Number of epochs used for training the RBM
                   'eta_rbm': {'values': [1e-4]},
                   'num_epochs_logistic': {'values': [100]},
                   'eta_logistic': {'values': [0.1]}
                  }
sweep_config['parameters'] = parameters_dict

In [5]:
#####################################
# Defining the train function
#####################################
def RBM_wandb_logs(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'numHiddenVars_{}_k_{}_r_{}'.format(config.num_hidden_vars, \
                          config.k, config.r)
        
        ###########################################
        # Training a classifier using RBM
        ###########################################
        RBM_mod = RBM(num_visible = config.num_visible_vars, num_hidden = config.num_hidden_vars)
        #test_acc_hist = []

        for epoch_num in tqdm(range(config.num_epochs_rbm)):
            # Training the RBM for one epoch
            RBM_mod.train(train_type = config.rbm_train_type, input_data = X_train, k = config.k, \
                          r = config.r, eta = config.eta_rbm)
            
            # Getting hidden representations of the validation data and test data
            X_val_hidden = RBM_mod.get_hidden_rep(X_val)
            X_test_hidden = RBM_mod.get_hidden_rep(X_test)
            
            # Training a logistic classifier on the validation data 
            logistic_clf = LogisticClassifier()
            
            logistic_clf.fit(X_val_hidden, y_val, epochs = config.num_epochs_logistic, eta = config.eta_logistic)
            
            # Evaluating the classifier on the validation data and test data
            _, val_acc, val_loss = logistic_clf.predict(X_val_hidden, y_val)
            _, test_acc, test_loss = logistic_clf.predict(X_test_hidden, y_test)

            '''
            # Early stopping
            if epoch_num >= config.patience_rbm and test_acc_hist[epoch_num-config.patience_rbm] > test_acc:
                break
            test_acc_hist.append(test_acc)
            '''

            # Logging metrics to wandb
            wandb.log({'val_loss': val_loss, 'val_acc': val_acc, \
                       'test_loss': test_loss, 'test_acc': test_acc})
            

In [ ]:
#################################
# Setting up wandb sweeps
#################################
sweep_id = wandb.sweep(sweep_config, project = 'DL-Assignment4-24May-2')
wandb.agent(sweep_id, function = RBM_wandb_logs)

Create sweep with ID: bisnhtr7
Sweep URL: https://wandb.ai/cs6910-team/DL-Assignment4-24May-2/sweeps/bisnhtr7


wandb: Agent Starting Run: 03nz3bo0 with config:
wandb: 	eta_logistic: 0.1
wandb: 	eta_rbm: 0.0001
wandb: 	k: 100
wandb: 	num_classes: 10
wandb: 	num_epochs_logistic: 100
wandb: 	num_epochs_rbm: 10
wandb: 	num_hidden_vars: 64
wandb: 	num_visible_vars: 784
wandb: 	r: 10
wandb: 	rbm_train_type: BGS
